<a href="https://colab.research.google.com/github/srujith7/AD-075/blob/main/12_feb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
data=pd.read_csv("/content/fake_news.csv", encoding='latin-1') # or 'ISO-8859-1', 'cp1252'
data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didnât Even See Comeyâs...,Darrell Lucus,House Dem Aide: We Didnât Even See Comeyâs...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [6]:
data=data.drop(['id'],axis=1)

In [7]:
data=data.fillna('')

In [8]:
data['content']=data['author']+' '+data['title']+' '+data['text']

In [9]:
data=data.drop(['author','title','text'],axis=1)

In [10]:
data['content']=data['content'].apply(lambda x: " ".join(x.lower() for x in x.split()))

In [ ]:
data['content']=data['content'].str.replace('[^\w\s]','')

In [11]:
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn import metrics

In [12]:
x=data[['content']]
y=data['label']

In [13]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=45,stratify=y)

In [14]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(14560, 1)
(6240, 1)
(14560,)
(6240,)


In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf=TfidfVectorizer()

In [16]:
tfidf_vect=TfidfVectorizer(analyzer='word',token_pattern=r'\w{1,}',max_features=5000)
tfidf_vect.fit(data['content'])
x_train_tfidf=tfidf_vect.transform(x_train['content'])
x_test_tfidf=tfidf_vect.transform(x_test['content'])

In [17]:
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn import metrics
pclf=PassiveAggressiveClassifier()
pclf.fit(x_train_tfidf,y_train)
predictions=pclf.predict(x_test_tfidf)
print(metrics.classification_report(y_test,predictions))

              precision    recall  f1-score   support

           0       0.96      0.98      0.97      3116
           1       0.98      0.96      0.97      3124

    accuracy                           0.97      6240
   macro avg       0.97      0.97      0.97      6240
weighted avg       0.97      0.97      0.97      6240



In [18]:
print(metrics.confusion_matrix(y_test,predictions))

[[3054   62]
 [ 140 2984]]


In [19]:
# Corrected import statement
from sklearn.neural_network import MLPClassifier

mlpclf = MLPClassifier(hidden_layer_sizes=(256, 64,16),activation = 'relu', solver = 'adam')
mlpclf.fit(x_train_tfidf,y_train)
predictions=mlpclf.predict(x_test_tfidf)
print(metrics.classification_report(y_test,predictions))

              precision    recall  f1-score   support

           0       0.96      0.97      0.97      3116
           1       0.97      0.96      0.97      3124

    accuracy                           0.97      6240
   macro avg       0.97      0.97      0.97      6240
weighted avg       0.97      0.97      0.97      6240



In [20]:
print(metrics.confusion_matrix(y_test,predictions))

[[3023   93]
 [ 122 3002]]


In [ ]:
import pickle
pickle.dump(mlpclf, open('model.pkl', 'wb'))

In [21]:
def fake_news_det(news):
  input_data=[news]
  vectorized_input_data=tfidf_vect.transform(input_data)
  prediction=pclf.predict(vectorized_input_data)
  print(prediction)

In [22]:
fake_news_det('U.S. Secretary of State John F. Kerry said Monday that he will stop in Paris later this week, amid criticism that no top American officials attended Sundayâ€™s unity march against terrorism.')

[1]


In [23]:
fake_news_det('U.S. Secretary of State John F. Kerry said Monday that he will stop in Paris later this week, amid criticism that no top American officials attended Sundayâ€™s unity march against terrorism.')

[1]


In [24]:
fake_news_det("""Go to Article
President Barack Obama has been campaigning hard for the woman who is supposedly going to extend his legacy four more years. The only problem with stumping for Hillary Clinton, however, is sheâ€™s not exactly a candidate easy to get too enthused about.  """)

[1]


In [25]:
!pip install flask